## Frog on Lilypad

In [16]:
#import section - use the abstract Markov Process class from rl directory
from rl.markov_decision_process import FiniteMarkovDecisionProcess, StateActionMapping, FinitePolicy
from rl.distribution import Categorical, Constant
from typing import Dict, Tuple
import numpy as np
import math

#### Problem 3.2
Assumption: user-defined n is between 2 and 100. Action representation: 0 = "A", 1 = "B".

In [17]:
class FrogEscape(FiniteMarkovDecisionProcess[int,int]):
    
    def __init__(self,n:int):
        self.n = max(min(n,100),2)
        super().__init__(self.get_action_transition_reward_map())
        
    def get_action_transition_reward_map(self) -> StateActionMapping[int,int]:
        d: Dict[int,Dict[str,Categorical[Tuple[int,float]]]] = {}
            
        d[0] = None
        d[self.n] = None
        
        for s in range(1,self.n):
            a: Dict[str,Categorical[Tuple[int,float]]] = {}
            a[0] = Categorical({(s-1,-1.):(float(s)/float(self.n)),
                               (s+1,1.):(1-float(s)/float(self.n))})
            a[1] = Categorical({(i,float(i-s)):(1./self.n)
                                 for i in range(self.n+1)})
            d[s] = a
            
        return d

For the below-used of algorithm, see "Theoretical-Hand". 

In [18]:
def optimal_deterministic(n:int) -> Tuple[np.ndarray, FinitePolicy[int,int]]:
    fe: FrogEscape = FrogEscape(n = n)
    
    totalNode: int = np.array([2**i for i in range(1,n)]).sum()
    leaveStart: int = totalNode - 2**(n-1)
        
    optVF: np.ndarray = np.repeat(-np.inf,n-1)
    optPolicy: FinitePolicy[int,int] = None
    
    for leaves in range(leaveStart,totalNode):
        actionList: list = [None]
        if leaves % 2 == 0:
            actionList.append(0)
        else:
            actionList.append(1)
        for _ in range(n-2):
            if (math.floor(leaves/2)-1) % 2 == 0:
                actionList.append(0)
            else:
                actionList.append(1)
            leaves = math.floor(leaves/2)-1
        actionList.append(None)
        
        currPolicy: FinitePolicy[int,int] = FinitePolicy({i:Constant(actionList[i]) for i in range(n+1)})
        
        implied_fe: FiniteMarkovRewardProcess[int] = fe.apply_finite_policy(currPolicy)
        currVF: np.ndarray = implied_fe.get_value_function_vec(gamma = 1.)
            
        if np.all(currVF > optVF):
            optVF = currVF
            optPolicy = currPolicy
    
    if optPolicy == None:
        return
    else:
        return optVF,optPolicy

In [19]:
#Test Correctness
optimal_deterministic(n = 3)

(array([0.71428571, 0.14285714]),
 For State 0:
   Do Action None with Probability 1.000
 For State 1:
   Do Action 1 with Probability 1.000
 For State 2:
   Do Action 0 with Probability 1.000
 For State 3:
   Do Action None with Probability 1.000)

In [20]:
#Test Correctness
optimal_deterministic(n = 6)

(array([ 2.95744681,  2.21276596,  1.34042553,  0.46808511, -0.27659574]),
 For State 0:
   Do Action None with Probability 1.000
 For State 1:
   Do Action 1 with Probability 1.000
 For State 2:
   Do Action 0 with Probability 1.000
 For State 3:
   Do Action 0 with Probability 1.000
 For State 4:
   Do Action 0 with Probability 1.000
 For State 5:
   Do Action 0 with Probability 1.000
 For State 6:
   Do Action None with Probability 1.000)

In [21]:
#Test Correctness
optimal_deterministic(n = 9)

(array([ 5.08108108,  4.34712838,  3.42314189,  2.46114865,  1.49155405,
         0.52956081, -0.39442568, -1.12837838]),
 For State 0:
   Do Action None with Probability 1.000
 For State 1:
   Do Action 1 with Probability 1.000
 For State 2:
   Do Action 0 with Probability 1.000
 For State 3:
   Do Action 0 with Probability 1.000
 For State 4:
   Do Action 0 with Probability 1.000
 For State 5:
   Do Action 0 with Probability 1.000
 For State 6:
   Do Action 0 with Probability 1.000
 For State 7:
   Do Action 0 with Probability 1.000
 For State 8:
   Do Action 0 with Probability 1.000
 For State 9:
   Do Action None with Probability 1.000)